In [1]:
import requests

from portfoliobuilder import api_utils, finnhub_endpoint, finnhub_key

### Notes for AAPL 2021 10-K
Year: response.json()['data'][0]['year']

Form: response.json()['data'][0]['form']

Report: response.json()['data'][0]['report']

Balance Sheet: response.json()['data'][0]['report']['bs']

Income Statement: response.json()['data'][0]['report']['ic']

Cash Flow Statement: response.json()['data'][0]['report']['cf']

Total Revenue: response.json()['data'][0]['report']['ic'][0]['value']

In [2]:
def get_fins_as_reported(symbol):
    url = finnhub_endpoint + 'stock/financials-reported'
    params = {'symbol': symbol, 'token': finnhub_key}
    return requests.get(url=url, params=params)

In [4]:
response = get_fins_as_reported('AAPL')
r_json = response.json()

In [7]:
# r_json['data'][0]['report']['ic']

for i in r_json['data'][0]['report']['ic']:
    print(i['label'])

Net income
Change in foreign currency translation, net of tax
Change in fair value of derivatives
Adjustment for net (gains)/losses realized and included in net income
Total change in unrealized gains/losses on derivative instruments
Change in fair value of marketable securities
Adjustment for net (gains)/losses realized and included in net income
Total change in unrealized gains/losses on marketable securities
Total other comprehensive income/(loss)
Other comprehensive income/(loss):
Net sales
Total cost of sales
Gross margin
Research and development expense
Selling, general and administrative
Operating expenses:
Operating income
Other income/(expense), net
Total amounts reclassified from AOCI
Provision for income taxes
Basic (in dollars per share)
Diluted (in dollars per share)
Basic (in shares)
Diluted (in shares)


## Testing Polygon financials endpoint

In [2]:
fins = api_utils.get_financials('AAPL')
fins = fins['results']

In [15]:
for key in fins[i].keys():
    print(key)
    # if key == 'reportPeriod':
    #     print(f'reportPeriod: {fins[i][key]}')

# print(fins[-1]['reportPeriod'])

ticker
period
calendarDate
reportPeriod
updated
dateKey
accumulatedOtherComprehensiveIncome
assets
assetsAverage
assetsCurrent
assetsNonCurrent
assetTurnover
bookValuePerShare
capitalExpenditure
cashAndEquivalents
cashAndEquivalentsUSD
costOfRevenue
consolidatedIncome
currentRatio
debtToEquityRatio
debt
debtCurrent
debtNonCurrent
debtUSD
deferredRevenue
depreciationAmortizationAndAccretion
deposits
dividendYield
dividendsPerBasicCommonShare
earningBeforeInterestTaxes
earningsBeforeInterestTaxesDepreciationAmortization
EBITDAMargin
earningsBeforeInterestTaxesDepreciationAmortizationUSD
earningBeforeInterestTaxesUSD
earningsBeforeTax
earningsPerBasicShare
earningsPerDilutedShare
earningsPerBasicShareUSD
shareholdersEquity
averageEquity
shareholdersEquityUSD
enterpriseValue
enterpriseValueOverEBIT
enterpriseValueOverEBITDA
freeCashFlow
freeCashFlowPerShare
foreignCurrencyUSDExchangeRate
grossProfit
grossMargin
goodwillAndIntangibleAssets
interestExpense
investedCapital
investedCapitalAver

## Rank stocks with ranking.py

In [2]:
from portfoliobuilder.ranking import get_measures

In [3]:
nasdaq100_stocks = api_utils.get_index_constituents('^NDX')

In [5]:
nasdaq100_stocks_measures = {}
for stock in nasdaq100_stocks[:20]:
    try:
        measures = get_measures(stock)
        nasdaq100_stocks_measures[stock] = measures
        print(f'success for {stock}')
    except (KeyError, IndexError):
        print(f'KeyError for {stock}')

KeyError for PTON
success for TSLA
success for REGN
success for CHTR
success for CSX
success for LRCX
success for GILD
success for CDW
success for NVDA
success for AMZN
success for AMAT
KeyError for GOOG
success for MDLZ
success for NFLX
success for TXN
success for MXIM
success for CHKP
success for ATVI
success for MTCH
success for BKNG


In [6]:
def rank(stocks_and_measures):
    symbols = stocks_and_measures.keys()
    
    # Rank according to valuation
    pe_ttm = []
    ev_ebitda = []
    p_fcf_last_year = []
    ev_fcf = []
    ps_ttm = []
    ev_s = []
    pb = []
    for symbol in stocks_and_measures:
        valuation_measures = symbol['valuation_measures']
        if valuation_measures['_ttm_ttm']:
            pe_ttm.append(valuation_measures['pe_ttm'])
        else:
            pe_ttm.append(100)

        if valuation_measures['ev_ebitda']:
            ev_ebitda.append(valuation_measures['ev_ebitda'])
        else:
            ev_ebitda.append(100)

        if valuation_measures['p_fcf_last_year']:
            p_fcf_last_year.append(valuation_measures['p_fcf_last_year'])
        else:
            p_fcf_last_year.append(100)

        if valuation_measures['ev_fcf']:
            ev_fcf.append(valuation_measures['ev_fcf'])
        else:
            ev_fcf.append(100)

        if valuation_measures['ps']:
            ps_ttm.append(valuation_measures['ps'])
        else:
            ps_ttm.append(100)

        if valuation_measures['ev_s']:
            ev_s.append(valuation_measures['ev_s'])
        else:
            ev_s.append(100)

        if valuation_measures['pb']:
            pb.append(valuation_measures['pb'])
        else:
            pb.append(100)

    total_pe_ttm = sum(pe_ttm)
    total_ev_ebitda = sum(ev_ebitda)
    total_p_fcf_last_year = sum(p_fcf_last_year)
    total_ev_fcf = sum(ev_fcf)
    total_ps_ttm = sum(ps_ttm)
    total_ev_s = sum(ev_s)
    total_pb = sum(pb)

    valuation_rankings = {} # symbol: {measure_name: measure_value}
    for symbol in stocks_and_measures:
        valuation_measures = stocks_and_measures['symbol']['valuation_measures']

        pe_rank = 1 - (valuation_measures['pe_ttm'] / total_pe_ttm)
        ev_ebitda_rank = 1 - (valuation_measures['ev_ebitda'] / total_ev_ebitda)
        p_fcf_last_year_rank = 1 - (valuation_measures['p_fcf_last_year'] / total_p_fcf_last_year)
        ev_fcf_rank = 1 - (valuation_measures['ev_fcf'] / total_ev_fcf)
        ps_ttm_rank = 1 - (valuation_measures['ps_ttm'] / total_ps_ttm)
        ev_s_rank = 1 - (valuation_measures['ev_s'] / total_ev_s)
        pb_rank = 1 - (valuation_measures['pb'] / total_pb)

        valuation_rankings[symbol] = {
            'pe_ttm': pe_rank,
            'ev_ebitda': ev_ebitda_rank,
            'p_fcf_last_year': p_fcf_last_year_rank,
            'ev_fcf_rank': ev_fcf_rank,
            'ps_ttm_rank': ps_ttm_rank,
            'ev_s_rank': ev_s_rank,
            'pb_rank': pb_rank
        }

    # Rank according to quality
    quality_measures_lists = {
        'profit_margin': [s['quality_measures']['profit_margin'] for s in stocks_and_measures],
        'revenue_growth': [s['quality_measures']['revenue_growth'] for s in stocks_and_measures],
        'ebitda_growth': [s['quality_measures']['ebitda_growth'] for s in stocks_and_measures],
        'roe_income': [s['quality_measures']['roe_income'] for s in stocks_and_measures],
        'roaa_income': [s['quality_measures']['roaa_income'] for s in stocks_and_measures],
        'roic_income': [s['quality_measures']['roic_income'] for s in stocks_and_measures],
        'roe_ebitda': [s['quality_measures']['roe_ebitda'] for s in stocks_and_measures],
        'roa_ebitda': [s['quality_measures']['roa_ebitda'] for s in stocks_and_measures],
        'roic_ebitda': [s['quality_measures']['roic_ebitda'] for s in stocks_and_measures],
        'current_ratio': [s['quality_measures']['current_ratio'] for s in stocks_and_measures],
        'assets_to_liabilities': [s['quality_measures']['assets_to_liabilities'] for s in stocks_and_measures]
    }
    # Handle negative numbers
    for measure in quality_measures_lists:
        smallest = min([i if i else 0.1 for i in quality_measures_lists[measure]])
        removed_nones = [i if i else smallest for i in quality_measures_lists[measure]]
        # TODO: add smallest to each value to handle negative numbers
    # TODO: Get totals
    # TODO: Calculate ranks




{'TSLA': {'valuation_measures': {'pe_ttm': 570.0569,
   'ev_ebitda': 73.169,
   'p_fcf_last_year': 0.0006500451181911614,
   'ev_fcf': None,
   'ps_ttm': 17.59861,
   'ev_s': 6.213021203678086,
   'pb': 27.47},
  'quality_measures': {'profit_margin': -3.39086,
   'revenue_growth': 50.78412,
   'ebitda_growth': None,
   'roe_income': 2.78868,
   'roaa_income': -0.027,
   'roic_income': -0.002,
   'roe_ebitda': 0.3632883935842434,
   'roa_ebitda': 0.06082951995103326,
   'roic_ebitda': 0.07067023120721051,
   'current_ratio': 1.66062,
   'assets_to_liabilities': 1.3095537997633497}},
 'REGN': {'valuation_measures': {'pe_ttm': 12.56928,
   'ev_ebitda': 15.728,
   'p_fcf_last_year': 2.84370725854829e-05,
   'ev_fcf': 15.19246,
   'ps_ttm': 6.18482,
   'ev_s': 5.339588796067859,
   'pb': 4.75495},
  'quality_measures': {'profit_margin': 30.84146,
   'revenue_growth': 15.66939,
   'ebitda_growth': 23.50349,
   'roe_income': 23.88211,
   'roaa_income': 0.155,
   'roic_income': 0.22,
   'roe_e